In [1]:
!pip install datasets
!pip install nltk wordcloud seaborn

In [18]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=8e5ff0ec04dbe53b10febe73d65f24ec80c858c21fb1aa86cc73a75b8419a9da
  Stored in directory: /Users/kaijimima1234/Library/Caches/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [11]:
from datasets import load_dataset
import pandas as pd

# 加载前 10000 条推文数据
dataset = load_dataset("momo1942/x_dataset_10290", split='train[:10000]')

# 转换为 DataFrame
df = pd.DataFrame(dataset)
print("数据结构预览：")
print(df.head())

Resolving data files:   0%|          | 0/414 [00:00<?, ?it/s]

train-DataEntity_chunk_482815_1.parquet:   0%|          | 0.00/103M [00:00<?, ?B/s]

train-DataEntity_chunk_482817_1.parquet:   0%|          | 0.00/128M [00:00<?, ?B/s]

train-DataEntity_chunk_482829_1.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [23]:
import pandas as pd
import re
from langdetect import detect
from tqdm import tqdm

tqdm.pandas()

# 移除非 ASCII 字符
def remove_non_ascii(text):
    return ''.join(c for c in text if 0 < ord(c) < 127)

# 清洗文本 + 保留英文
def clean_and_filter_english(text):
    if not isinstance(text, str):
        return None
    
    # 1. 去除非 ASCII 字符
    text = remove_non_ascii(text)
    
    # 2. 转小写
    text = text.lower()
    
    # 3. 移除链接、@提及、#
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)

    # 4. 移除特殊符号、数字，仅保留字母和空格
    text = re.sub(r"[^\sa-zA-Z]", "", text)

    # 5. 去掉多余空格
    text = re.sub(r"\s+", " ", text).strip()

    # 6. 语言检测（只保留英文）
    try:
        if detect(text) == 'en':
            return text
        else:
            return None
    except:
        return None

# ✅ 应用清洗和语言过滤
df['cleaned_text'] = df['text'].progress_apply(clean_and_filter_english)

# ✅ 删除非英文或空文本
df = df[df['cleaned_text'].notnull()]

# ✅ 时间列处理（转换为 datetime 对象 + 取日期）
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')  # 避免格式错误
df['date'] = df['datetime'].dt.date

# ✅ 查看结果样本
print("\n✅ 清洗与日期提取后的数据样本：")
print(df[['datetime', 'cleaned_text']].head(10))

100%|██████████████████████████████████| 100000/100000 [04:21<00:00, 382.40it/s]



✅ 清洗与日期提取后的数据样本：
     datetime                                       cleaned_text
0  2025-01-23                  x inhun squidgame two frontmen au
2  2025-01-23  his calm demeanour sharp golden gaze while sur...
5  2025-01-23  dont miss communiques investment opportunity l...
7  2025-01-23  c fall winter collection fashion show at paris...
11 2025-01-23  mexican navy cw anx db departed culiacn for tepic
14 2025-01-23                      x talking about ai and crypto
15 2025-01-23          nothing we cant do in brett eth x ogbrett
18 2025-01-23  smart money knows where to park xgas is tickin...
19 2025-01-23  smart money knows where to park xgas is tickin...
20 2025-01-23  xgas keeps delivering cant wait for your ai ag...


In [1]:
!pip install vaderSentiment yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for peewee: filename=peewee-3.18.1-cp312-cp312-macosx_11_0_arm64.whl size=264309 sha256=1773f92c8f1dbc3cf6ee86d71848ec36d83f00378afd2b6dabbecda5e01bcb72
  Stored in directory: /Users/kaijimima1234/Library/Caches/pip/wheels/1a/57/6a/bb71346381d0d911cd4ce3026f1fa720da76707e4f01cf27dd
Successfully built peewee


In [25]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

# 初始化 VADER
analyzer = SentimentIntensityAnalyzer()

# 情绪打分函数
def get_sentiment_score(text):
    if isinstance(text, str):
        score = analyzer.polarity_scores(text)
        return score['compound']  # 取复合情绪得分
    else:
        return 0

# 对 cleaned_text 计算情绪分数
df['sentiment'] = df['cleaned_text'].apply(get_sentiment_score)

# 按日期聚合情绪得分（平均值）
daily_sentiment = df.groupby('date')['sentiment'].mean().reset_index()
daily_sentiment.columns = ['date', 'avg_sentiment']

print("✅ 每日平均情绪得分：")
print(daily_sentiment.head())

✅ 每日平均情绪得分：
         date  avg_sentiment
0  2025-01-23       0.189732


In [ ]:
#加载ETH价格
import yfinance as yf

eth = yf.download("ETH-USD", start="2025-01-23", end="2025-02-20")
eth = eth[['Open', 'Close']]
eth['date'] = eth.index.date
eth.reset_index(drop=True, inplace=True)
eth.columns = ['eth_open', 'eth_close', 'date']

In [ ]:
#计算涨跌幅和标记趋势
eth['eth_return_pct'] = (eth['eth_close'] - eth['eth_open']) / eth['eth_open']
eth['eth_trend'] = eth['eth_return_pct'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

In [ ]:
#合并情绪和价格
merged = pd.merge(daily_sentiment_df, eth, on='date')

In [ ]:
#可视化对比
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(merged['date'], merged['avg_sentiment_score'], label='平均情绪分数', color='blue', marker='o')
plt.plot(merged['date'], merged['eth_return_pct'], label='ETH涨跌幅', color='green', marker='x')
plt.axhline(0, color='black', linestyle='--')
plt.legend()
plt.title("每日情绪分数 vs ETH 涨跌幅")
plt.xlabel("日期")
plt.ylabel("数值")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
#计算皮尔逊相关系数
correlation = merged['avg_sentiment_score'].corr(merged['eth_return_pct'])
print(f"情绪与ETH涨跌的相关系数为：{correlation:.4f}")